# 1. Model Problem
We deal with the following interface problem
$$
\begin{aligned}
          - \nabla \cdot (\alpha \nabla u) + cu  = & \, f 
          & & \text{in}~~ \Omega_{1} \cup \Omega_{2}, 
          \\
          [\![u]\!] = & \, 0 
          &  & \text{on}~~ \Gamma, 
          \\
          [\![-\alpha \nabla u \cdot \mathbf{n}]\!]   = & \, 0 
          &  & \text{on}~~ \Gamma,
          \\
          u = & \, u_D
          &  & \text{on}~~ \partial \Omega.
        \end{aligned} 
$$
where $\alpha$ is a piecewise constant date so that $\alpha_i = \alpha|_{\Omega_i}$.


# 2. Nitsche formulation
The method is defined by the variational problem of finding $U \in V^h$ such that 
$$ a_h(U,v)  = L(v), \qquad \forall v \in V^h, $$
where
$$
\begin{aligned}
        a_h(U,v) &:= \sum_{i} \left[ \left( \alpha_i \nabla U_i, \nabla v_i \right)_{\Omega_i} + (cu,v)_{\Omega_i} \right] - \left( [\![U]\!],\{\!\!\{  \alpha \nabla v \cdot n \}\!\!\} \right)_\Gamma - \left( \{\!\!\{  \alpha \nabla U \cdot n \}\!\!\}, [\![v]\!] \right)_\Gamma  + \left( \frac{\gamma}{h}  [\![U]\!] , [\![v]\!] \right)_{\Gamma} \\
         L(v) &:= \sum_{i}\left( f,v_i \right)_{\Omega_i}
\end{aligned}
$$

## 2.1 About the average operator:
- If the contrast of $\alpha$ in different region is relatively small: $\{\!\!\{  \alpha \nabla v \cdot n \}\!\!\}= (\alpha_1 \kappa_1 \nabla v_1 \cdot n + \alpha_2 \kappa_2 \nabla v_2 \cdot n)|_{\Gamma}$ with $\kappa_i|_K = \frac{\operatorname{meas}(K_i)}{\operatorname{meas}(K)} $.
- If the contrast of $\alpha$ in different region is large: use the weighted average by changing the definitions of the $\kappa_i$ to $\kappa_1|_K := \frac{\alpha_2 \operatorname{meas}(K_1)}{\alpha_2 \operatorname{meas}(K_1)+\alpha_1 \operatorname{meas}(K_2)}$, $\kappa_2|_K := \frac{\alpha_1 \operatorname{meas}(K_2)}{\alpha_2 \operatorname{meas}(K_1)+\alpha_1 \operatorname{meas}(K_2)}$.

## 2.2 About the stability parameter $\gamma$
For stability, $\gamma$ should be chosen sufficiently large. There are two proposed choices:
- $ \gamma |_K = \hat{\gamma} h_K \frac{m_{K_\Gamma}}{m_{K_{\Omega_1}} / \alpha_1 + m_{K_{\Omega_2}} / \alpha_2}$
- $ \gamma |_K = \hat{\gamma} h_K \max\left\{ \alpha_1 \frac{m_{K_{\Omega_1}}}{m_K}, \alpha_2 \frac{m_{K_{\Omega_2}}}{m_K} \right\} \frac{m_{K_\Gamma}}{m_K}$
  
Above $\hat{\gamma}$ is a mesh and parameter independent constant and
$$
m_{K_\Gamma} := \text{meas}_{d - 1}(K \cap \Gamma), \quad m_{K_{\Omega_i}} := \text{meas}_d(K \cap \Omega_i), \, i = 1, 2 \quad m_K := \text{meas}_d(K).
$$

In [1]:
from math import pi 
from ngsolve import *
from xfem import *
from netgen.occ import *
from ngsolve.webgui import *

importing ngsxfem-2.1.2504


In [2]:
# Important parameters and functions
alpha = [2, 1]
gamma = 1000
f = [-8+x**2+y**2, -8+2*(x**2+y**2)-1/4]
c = [1, 1]
gD = 2*(x**2+y**2)-1/4
exact_u_neg = x**2+y**2
exact_u_pos = 2*(x**2+y**2)-1/4

In [3]:
# Construct background mesh
geo = OCCGeometry(unit_square_shape.Scale((0,0,0),2).Move((-1,-1,0)), dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05,quad_dominated=False))

In [4]:
# Define the levelset function for the interface
levelset = sqrt(x**2+y**2)-0.5
lsetp1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(levelset,lsetp1)
DrawDC(lsetp1,-1,1,mesh,'lsetp1')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [5]:
# Construct the unfitted fem space 
Vhbase = H1(mesh,order=1,dirichlet='.*')
ci = CutInfo(mesh,lsetp1)
Vh = FESpace([Compress(Vhbase,GetDofsOfElements(Vhbase,ci.GetElementsOfType(cdt))) for cdt in [HASNEG, HASPOS]])

In [6]:
# Visualize the dofs
freedofs = Vh.FreeDofs()
gfu = GridFunction(Vh)
gfu.components[0].Set(1)
gfu.components[1].Set(-1)
DrawDC(lsetp1, gfu.components[0], gfu.components[1], mesh, "u")
print(Vhbase.ndof, Vh.components[0].ndof, Vh.components[1].ndof)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

1937 430 1648


In [7]:
print(len(freedofs))
dofs = GetDofsOfElements(Vhbase, ci.GetElementsOfType(IF)) # 与界面相交的单元包含的自由度
print(dofs.NumSet())

2078
141


In [8]:
# Define averages and jumps

## Get cut ratio kappas
kappaminus = CutRatioGF(ci)
kappa = (kappaminus, 1-kappaminus)
# print(help(kappaminus))
# Draw(kappaminus,mesh)

## averages
n = Normalize(grad(lsetp1)) # normal vector on interface
h = specialcf.mesh_size

u,v = Vh.TnT()
average_u = sum([kappa[i] * alpha[i] * grad(u[i]) * n for i in range(2)])
average_v = sum([kappa[i] * alpha[i] * grad(v[i]) * n for i in range(2)]) 
jump_u = u[0] - u[1]
jump_v = v[0] - v[1]


In [9]:
print(type(kappaminus))

<class 'ngsolve.comp.GridFunction'>


In [10]:
# Construc bilinear form and right hand side 

## integral operators
dx_neg = dCut(levelset=lsetp1, domain_type = NEG, definedonelements=ci.GetElementsOfType(HASNEG))
dx_pos = dCut(levelset=lsetp1, domain_type = POS, definedonelements=ci.GetElementsOfType(HASPOS))
ds = dCut(levelset=lsetp1, domain_type = IF, definedonelements=ci.GetElementsOfType(IF))

## Bilinear form
ah = BilinearForm(Vh,symmetric=True)
ah += (alpha[0] * grad(u[0]) * grad(v[0]) * dx_neg + alpha[1] * grad(u[1]) * grad(v[1]) * dx_pos)
ah += (c[0] * u[0] * v[0] * dx_neg + c[1] * u[1] * v[1] * dx_pos)
ah += -(jump_u * average_v  + jump_v * average_u - gamma / h * jump_u * jump_v) * ds
ah. Assemble()

## right hand side
F = LinearForm(Vh)
F += ( f[0] * v[0] * dx_neg + f[1] * v[1] * dx_pos )
F.Assemble()


In [11]:
# Solve the system

gfu = GridFunction(Vh)
## deal with Dirichlet boundary
gfu.components[1].Set(gD,BND)
F.vec.data -= ah.mat * gfu.vec
gfu.vec.data += ah.mat.Inverse(freedofs) * F.vec


In [12]:
DrawDC(lsetp1, gfu.components[0], gfu.components[1], mesh, "u")
DrawDC(lsetp1, exact_u_neg, exact_u_pos, mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [13]:
# Calculate L2 error
# u_exact = [exact_u_neg, exact_u_pos]
err = sqrt(Integrate((gfu.components[0] - exact_u_neg)**2*dx_neg, mesh) + Integrate((gfu.components[1] - exact_u_pos)**2*dx_pos, mesh))

In [14]:
print(err)

0.0014492244711278401
